In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
import pandas as pd
#from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import applications
#from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping


In [ ]:
#applying transfer learning
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (224, 224, 3))

In [ ]:
# Freeze first 5 layers.
for layer in model.layers[:5]:
    layer.trainable = False

In [ ]:
# adding layers
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(62, activation="softmax")(x)

In [ ]:
model_final = Model(input = model.input, output = predictions)

In [ ]:
model_final.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=["accuracy"])

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range = 30,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    directory="img/train/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=16,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

valid_generator = test_datagen.flow_from_directory(
    directory="img/valid/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=16,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    directory="img/test/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

In [ ]:
checkpoint = ModelCheckpoint("vgg19_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

model_final.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=30,
                    callbacks = [checkpoint, early]
)

In [ ]:
model_final.save('image_model.hdf5')

In [ ]:
model_json = model_final.to_json()
with open("model_image.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
model_final.save_weights("image_model_weights.h5")
print("Saved model to disk")

In [ ]:
model_final.save_weights("vgg16_1.h5")
print("Saved model to disk")

In [ ]:
from keras.models import model_from_json

In [ ]:
json_file = open('model_image.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [ ]:
# load weights into new model
loaded_model.load_weights("image_model_weights_2.h5")
print("Loaded model from disk")

In [ ]:
test_generator.reset()
pred=loaded_model.predict_generator(test_generator,verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})

In [ ]:
results

In [ ]:
results[['directory', 'file']] = results.Filename.str.split("\\", expand = True)

In [ ]:
results.to_csv('test_keras.csv', index = False)

In [ ]:
(results.Predictions == results.directory).mean()

In [ ]:
from PIL import Image
from skimage import transform

In [ ]:
np_image = Image.open('img/test/ADELLE BAG/ADELLE BAG_180304_0031.JPG')

In [ ]:
test_image = np.array(np_image)/255
test_image = transform.resize(test_image, (3,150,150))
#test_image = test_image.reshape(test_image.shape[2], test_image.shape[0], test_image.shape[1])

In [ ]:
test_image = np.expand_dims(test_image, axis = 0)
test_list = loaded_model.predict(test_image).tolist()

In [ ]:
loaded_model.predict_classes(test_list)

In [ ]:
def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (3, 150, 150))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image

image = load('img/test/ADELLE BAG/ADELLE BAG_180304_0031.JPG')

In [ ]:
test_list = loaded_model.predict(image).tolist()

In [ ]:
probs = []
for i in test_list:
    for j in i:
        probs.append(j)

In [ ]:
test_labels = list(labels.values())

In [ ]:
test_df = pd.DataFrame({'label': test_labels, 'probs': probs})

In [ ]:
test_list

In [ ]:
test_df.sort_values('probs', ascending = False)

In [ ]:
test_df.sort_values('probs', ascending = False)

## Test 1: Adam
Epoch = 10, loss = categorical_crossentropy, metrics = accuracy  
loss: 0.9872 - acc: 0.7294 - val_loss: 3.0550 - val_acc: 0.406

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20
)